In [1]:
# import libraries
import os
import sagemaker
import boto3
import pandas as pd
import ast
import numpy as np
import copy
import json
import argparse
import torch
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorch as PyTorchEstimator

In [3]:
#import sagemaker session and role, as well as S3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket='arn:aws:s3:::circ-rna'
%store bucket

Stored 'bucket' (str)


In [4]:
# set parameters
hyperparameters={
        "train_data_dir": 's3://circ-rna/data/circRNA_lncRNA_train.csv',
        "test_data_dir": 's3://circ-rna/data/circRNA_lncRNA_test.csv',
        "vocab_dir": 's3://circ-rna/data/vocab.csv',
        "batch_size": 16,
        "epochs":10,
    }

In [5]:
# pygmentize train.py
!pygmentize "source_dir/train.py"

# import libraries
import math
import argparse
import collections
import copy
import os
import json
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, dataloader
from sklearn import metrics
from torch.nn  import functional as F
import torch.optim as  optim
from tqdm import tqdm
import random
import ast
from sklearn import metrics
import logging
import sys
from sklearn.metrics import accuracy_score
from sklearn.metrics import  f1_score
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
CUDA_LAUNCH_BLOCKING=1

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

# Define the padding function to pad all sequences to max length
def pad_sequences(seqs,max_length=400,unk_index=64):
    pad_seqs=[]
  

In [8]:
estimator = PyTorchEstimator(
    entry_point='source_dir/train.py',
    #source_dir='s3://sagemaker-us-east-2-411668307327/circRNA/',
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=50,
    hyperparameters=hyperparameters,
    #metric_definitions=metric_definitions,
    framework_version='1.7',
    py_version='py36',
)

In [9]:
channels = {
    'training': "s3://circ-rna/data/circRNA_lncRNA_train.csv",
    'test': "s3://circ-rna/data/circRNA_lncRNA_test.csv"
}
estimator.fit(channels)

2022-06-22 19:54:58 Starting - Starting the training job...
2022-06-22 19:55:23 Starting - Preparing the instances for trainingProfilerReport-1655927698: InProgress
.........
2022-06-22 19:56:41 Downloading - Downloading input data...
2022-06-22 19:57:21 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-22 19:58:09,338 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-22 19:58:09,340 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-22 19:58:09,353 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-22 19:58:09,359 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-22 19:58:09,751 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-22 19:58:09,768 sagemaker-training-t

In [10]:
model_data = estimator.model_data
print(model_data)

s3://sagemaker-us-east-2-411668307327/pytorch-training-2022-06-22-19-54-57-452/output/model.tar.gz


In [34]:
pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-east-2-411668307327/pytorch-training-2022-06-22-19-54-57-452/output/model.tar.gz',
                             role=role,
                             framework_version="1.7",
                             #source_dir="code",
                             py_version="py36",
                             entry_point="source_dir/train.py")
pytorch_model

In [35]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.t2.medium",serializer=JSONSerializer(), deserializer=JSONDeserializer())

--------------!

In [36]:
predictor

In [38]:
seq="GCATGTTGGCATTGAACATTGACGAAGCTATTACATTGCTTGAACAATTGGGACTTAGTGGCAGCTATCAATGGTGTAATACCACAGGATGGCATTCTACAAAGTGAATATGGAGGTGAGACCATACCAGGACCTGCATTTAATCCAGCAAGTCATCCAGCTTCAGCTCCTACTTCCTCTTCTTCTTCAGCGTTTCGACCTGTAATGCCATCCAGGCAGATTGTAGAAAGGCAACCTCGGATGCTGGACTTCAGGGTTGAATACAGAGACAGAAATGTTGATGTGGTACTTGAAGACACCTGTACTGTTG"
result = predictor.predict({"text": seq})
print(result)

1.0
